In [ ]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
from dotenv import load_dotenv
import os
import requests
from urllib.parse import quote

# Load environment variables from .env file
load_dotenv()
RIOT_API_KEY = os.getenv('RIOT_API_KEY')

# Verify API key is loaded
if RIOT_API_KEY is None:
    raise ValueError("RIOT_API_KEY not found in .env file! Please check your .env file.")
else:
    print(f"API Key loaded: {RIOT_API_KEY[:10]}...{RIOT_API_KEY[-5:]} (showing first 10 and last 5 chars)")

# Initialize the LolWatcher with your API key
watcher = LolWatcher(RIOT_API_KEY)

API Key loaded: RGAPI-cf28...0c663 (showing first 10 and last 5 chars)


In [ ]:
# Set Riot-ID Information
game_name = 'euclidean aatrox'  
tag_line = 'EUW' 
routing_region = 'europe'  
my_region = 'euw1'

In [87]:
# URL encode the game name (spaces become %20)
encoded_game_name = quote(game_name)
encoded_tag_line = quote(tag_line)

# Construct URL to find out PUUID (Summoner ID that is used in most APIs)
get_puuid_url = f"https://{routing_region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{encoded_game_name}/{encoded_tag_line}"

# Make request
headers = {
    "X-Riot-Token": RIOT_API_KEY
}

response = requests.get(get_puuid_url, headers=headers)

# Extract puuid from response
puuid = response.json()['puuid']
print(response.json())
print(puuid)


{'puuid': 'mRSUINe-WZCF2tn9ux-C0YrVUr3EDhHIcSpMildFEwd2aVC7WEMctiLo6K2g4NdcSG96ofbQBXE00w', 'gameName': 'euclidean aatrox', 'tagLine': 'EUW'}
mRSUINe-WZCF2tn9ux-C0YrVUr3EDhHIcSpMildFEwd2aVC7WEMctiLo6K2g4NdcSG96ofbQBXE00w


In [88]:
matches = watcher.match.matchlist_by_puuid(my_region, puuid)
last_match = matches[0]
print(last_match)

EUW1_7299723064


In [ ]:
match = watcher.match.by_id(my_region, last_match)
print(type(match))
print(f"Keys of match: {match.keys()}")
print(f"Keys of info (attributes for feature engineering): {match['info'].keys()}")
print(f"Values of info: {match['info'].values()}")
print(f"Keys of metadata: {match['metadata'].keys()}")
print(f"Values of metadata: {match['metadata'].values()}")
print(match)

<class 'dict'>
Keys of match: dict_keys(['metadata', 'info'])
Keys of info (attributes for feature engineering): dict_keys(['endOfGameResult', 'gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])
Values of info: dict_values(['GameComplete', 1739053346387, 1029, 1739054400796, 7299723064, 'URF', 'teambuilder-match-7299723064', 1739053371061, 'MATCHED_GAME', '15.3.654.407', 11, [{'PlayerScore0': 0, 'PlayerScore1': 0, 'PlayerScore10': 0, 'PlayerScore11': 0, 'PlayerScore2': 0, 'PlayerScore3': 0, 'PlayerScore4': 0, 'PlayerScore5': 0, 'PlayerScore6': 0, 'PlayerScore7': 0, 'PlayerScore8': 0, 'PlayerScore9': 0, 'allInPings': 0, 'assistMePings': 1, 'assists': 3, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 4, 'champExperience': 17239, 'champLevel': 17, 'championId': 80, 'championName': 'Pantheon', 'championTransform': 0, 'commandPing

In [ ]:
# FEATURE SELECTION - Selected attributes that contribute to win or lose

selected_attributes = [
    # Participant basic info
    'championName',
    #'champLevel',
    #'champExperience',
    'summonerName',
    
    # Win/Loss (target variable)
    'win',
    
    # Game outcome
    'gameEndedInSurrender',
    'gameEndedInEarlySurrender',
    
    # KDA
    'kills',
    'deaths',
    'assists',
    
    # Gold
    'goldEarned',
    'goldSpent',
    
    # CS (Creep Score)
    'totalMinionsKilled',
    #'neutralMinionsKilled',
    
    # Objectives
    'baronKills',
    'dragonKills',
    'turretKills',
    'turretTakedowns',  # From challenges
    'turretsLost',
    'inhibitorKills',
    'inhibitorTakedowns',
    'inhibitorsLost',
    'objectivesStolen',
    'objectivesStolenAssists',
    
    # First events
    'firstBloodKill',
    'firstBloodAssist',
    'firstTowerKill',
    'firstTowerAssist',
    
    # Damage
    #'totalDamageDealt',
    'totalDamageDealtToChampions',
    #'totalDamageTaken',
    #'damageDealtToBuildings',
    #'damageDealtToObjectives',
    #'damageDealtToTurrets',
    #'damageSelfMitigated',
    'damagePerMinute',  # From challenges
    
    # Healing & CC
    'totalHeal',
    'totalTimeCCDealt',
    'timeCCingOthers',
    
    # Vision
    'wardsPlaced',
    'wardsKilled',
    
    # Combat stats
    'killingSprees',
    #'largestKillingSpree',
    #'largestMultiKill',
    'longestTimeSpentLiving',
    'totalTimeSpentDead',
    'bountyLevel',
    
    # Spells
    'spell1Casts',
    'spell2Casts',
    'spell3Casts',
    'spell4Casts',
    
    # Other
    #'consumablesPurchased',
    'timePlayed',
    'challenges'  # Container object (specific challenge attributes extracted separately)
]

print(f"Total selected attributes: {len(selected_attributes)}")
print(f"\nGrouped attributes:")
print(f"  Participant basic info: 4")
print(f"  Win/Loss: 1")
print(f"  Game outcome: 2")
print(f"  KDA: 3")
print(f"  Gold: 2")
print(f"  CS: 2")
print(f"  Objectives: 9")
print(f"  First events: 4")
print(f"  Damage: 8")
print(f"  Healing & CC: 3")
print(f"  Vision: 2")
print(f"  Combat stats: 6")
print(f"  Spells: 4")
print(f"  Other: 3")
print(f"  Challenges container: 1")

Total selected attributes: 42

Grouped attributes:
  Participant basic info: 4
  Win/Loss: 1
  Game outcome: 2
  KDA: 3
  Gold: 2
  CS: 2
  Objectives: 9
  First events: 4
  Damage: 8
  Healing & CC: 3
  Vision: 2
  Combat stats: 6
  Spells: 4
  Other: 3
  Challenges container: 1


In [ ]:
# Get all match_ids to use that to retrieve match data (API limits only 100 match_ids possible at a time)
def get_all_ranked_match_ids(watcher, region, puuid):
    """Retrieve all ranked match IDs (100 per request max)"""
    all_match_ids = []
    start = 0
    
    while True:
        match_ids = watcher.match.matchlist_by_puuid(region, puuid, start=start, count=100)
        if not match_ids:
            break
        all_match_ids.extend(match_ids)
        if len(match_ids) < 100:
            break
        start += 100
    
    return all_match_ids

# Usage:
all_match_ids = get_all_ranked_match_ids(watcher, routing_region, puuid)
print(len(all_match_ids))

938


In [92]:
# Investigate data structure: one match has 10 entries for each player (5 vs 5)
unique_match_ids = df['matchId'].unique()
print(f"Number of unique match IDs: {len(unique_match_ids)}")

Number of unique match IDs: 50


In [93]:
# Iterate through all 938 matches to retrieve selected attributes using match.by_id 
def get_match_dataframe(watcher, region, match_ids, selected_attributes):
    """
    Extract match data into DataFrame using selected attributes.
    
    Parameters:
    -----------
    watcher : LolWatcher
        Initialized RiotWatcher instance
    region : str
        Routing region (europe, americas, asia)
    match_ids : list
        List of match IDs to process
    selected_attributes : list
        List of attribute names to extract from participant data
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with one row per participant, containing selected attributes
    """
    participants_data = []
    
    # Attributes that are nested inside 'challenges' object
    challenges_attrs = ['turretTakedowns', 'soloKills', 'damagePerMinute']
    
    print(f"Processing {len(match_ids)} matches...")
    
    for idx, match_id in enumerate(match_ids):
        try:
            # Fetch match data from API
            match_data = watcher.match.by_id(region, match_id)
            match_info = match_data['info']
            
            # Process each participant (10 per match)
            for participant in match_info['participants']:
                row = {'matchId': match_id}  # Always include matchId
                
                # Extract each selected attribute
                for attr in selected_attributes:
                    if attr == 'challenges':
                        # Skip 'challenges' itself, we extract specific challenge attributes
                        continue
                    elif attr in challenges_attrs:
                        # Extract from nested challenges object
                        row[attr] = participant.get('challenges', {}).get(attr)
                    else:
                        # Extract directly from participant object
                        row[attr] = participant.get(attr)
                
                participants_data.append(row)
            
            # Progress indicator every 100 matches
            if (idx + 1) % 100 == 0:
                print(f"  Processed {idx + 1}/{len(match_ids)} matches ({len(participants_data)} participants so far)")
                
        except ApiError as e:
            print(f"  API Error for match {match_id}: {e}")
            continue
        except Exception as e:
            print(f"  Error processing match {match_id}: {e}")
            continue
    
    # Create DataFrame
    df = pd.DataFrame(participants_data)
    
    print(f"\n✅ Complete! Created DataFrame with {len(df)} rows from {len(match_ids)} matches")
    print(f"   Shape: {df.shape} (rows × columns)")
    
    return df

# Process ALL match IDs with selected attributes
print("="*70)
print("EXTRACTING DATA FROM ALL MATCHES")
print("="*70)
df_all_matches = get_match_dataframe(watcher, routing_region, all_match_ids, selected_attributes)

print(f"\nDataFrame Info:")
print(f"  Total rows: {len(df_all_matches)}")
print(f"  Total columns: {len(df_all_matches.columns)}")
print(f"  Expected: ~{len(all_match_ids) * 10} rows (10 participants per match)")
print(f"\nFirst few rows:")
print(df_all_matches.head())
print(f"\nColumn names: {list(df_all_matches.columns)}")


EXTRACTING DATA FROM ALL MATCHES
Processing 938 matches...
  API Error for match EUW1_7223256757: 429 Client Error: Too Many Requests for url: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7223256757
  Processed 100/938 matches (990 participants so far)
  Processed 200/938 matches (1990 participants so far)
  Processed 300/938 matches (2990 participants so far)
  Processed 400/938 matches (4008 participants so far)
  Processed 500/938 matches (5038 participants so far)
  Processed 600/938 matches (6038 participants so far)
  Processed 700/938 matches (7038 participants so far)
  Processed 800/938 matches (8038 participants so far)
  Processed 900/938 matches (9038 participants so far)

✅ Complete! Created DataFrame with 9418 rows from 938 matches
   Shape: (9418, 42) (rows × columns)

DataFrame Info:
  Total rows: 9418
  Total columns: 42
  Expected: ~9380 rows (10 participants per match)

First few rows:
           matchId championName summonerName   win  gameEndedInSurre

In [94]:
df_all_matches.to_excel('all_matches_data.xlsx', index=False, engine='openpyxl')